# Multiple Choice Grader with OpenCV.

Hi folks, This is another interesting episode of the tutorial using opencv. In this episode, we are going to grade multiple choice questions using OpenCV and python. The idea is indeed simple. 

 1) we detect the four image itself and obtain the top-down view of the image like that of a properly scanned one. I have explained this in my previous tutorial, it will be implemented here again.

2) Having detected the four coners of the image, as well as obtaining a scanned view, we locate the positions of all the options and arrange them per question, that is 5 options to 1 question.

3) Next we detect the shaded option by looping through the options to see which has a different background colour due to shading.

4) Lastly, and finally, we score by confirming if the shaded option matches the answer key.

###  Let's code.

######  Import packages.
first and as usual, we import the packages we need, if you are new, the packages that might be quite new to you are 1) The OpenCV package: The package was originally developed by Intel and also contributed to by quite some special people, it is a very important tool in exploring images in python, as well as computer Vision. 2) Thanks to adrian Rosbrock, he created the imutils package which includes sets of convienience functions that are complicated using the opencv package.

In [1]:
# import the necessary packages
import numpy as np 
import imutils
import cv2
from imutils import contours as contour_manager

###### Answer key
Here, i just gave the answer key, starting from question 1 which corresponds to 0, in my answer key and the corresponding option.( A is 0, B is 1, C is 2, D is 3 and E is 4 ). so what i have doen here is (question1: option A, qustion 2: option B,......e.t.c)

In [2]:
# Write the expected answer key where 0 is the first question..... e.t.c
ANSWER_KEY = {0:0, 1:1, 2:1, 3:0, 4:3, 5:4, 6:4, 7:0, 8:1, 9:0}

###### Load in the image.
next, we load in the image, convert it to a grayscale, blurr the grayscaled image, then use canny to detect the edges of the paper.

In [3]:
# Load the input image 
image = cv2.imread("victor_script.JPG")
image  = cv2.resize(image,(416*2,312*2))
orig_image = image.copy()
# next we grayscale the image.
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Next step is to blur the image.
blurred = cv2.GaussianBlur(gray, (5,5), 0)
# Find edges using the canny edge detection. Make sure you blurr before finding edges.
edged = cv2.Canny(blurred, 70,200)

In [4]:
image.shape

(624, 832, 3)

###### Quick peek.
just to see all found edges in the image, as well as the read in image.

In [7]:
#cv2.imshow(" blurred  image", blurred)
cv2.imshow(" original image", image)
cv2.imshow(" edges of image", edged)
cv2.imshow(" gray-scale of image", gray)
cv2.imshow(" blurred gray-scale of image", blurred)
cv2.waitKey(0)

-1

######  Find Document Edge.
Although, i have properly explained and used this step in some of my previous tutorials, the idea here is to find all the contours, sort the contours by area, then approximate the contour shape. The intution is that the largest rectangular contour is most likely the document edge.

In [9]:
# FIND CONTOURS
contours = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# grab contours with imutils
contours = imutils.grab_contours(contours)
# our major interest here is the document contour, initialize to none.
#doc_contour = None

# sort all found contours by area in decending order.
if  len(contours) > 0:
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
# approximate contours and choose contour with exactly 4 sides as the edge.
for contour in contours:
    perimeter = cv2.arcLength(contour, True) # convert contour points to lines.
    approximated = cv2.approxPolyDP(contour, 0.02*perimeter, True) #Approximate these lines to shape
    if len(approximated) == 4: # if the approximated shape has 4 sides.
        doc_contour = approximated  # The document contour 
        break

######  Quick Peek.
Draw the contour you have proposed to be the document, and view it.

In [10]:
# draw the document edges.
cv2.drawContours(image,[doc_contour], -1, (255,0,0), 2)
# preview drawn contours.
cv2.imshow("found document edges",image)
cv2.imwrite("output4.jpg", image)
cv2.waitKey(0)

-1

###### Functions To Give The Birds Eye View.
This two functions help to obtain the birds eye view of an image with the four coners already detected. I have explained this explicitly in my previous tutorial.

In [13]:
def arrange_points(points):
    # initialize a list of co-ordinates that will be ordered
    # first entry is top-left point, second entry is top-right
    # third entry is bottom-right, forth/last point is the bottom left point.
    rectangle = np.zeros((4,2), dtype = "float32")
    
    # bottom left point should be the smallest sum
    # the top-right point will have the largest sum of point.
    sum_points= points.sum(axis =1)
    rectangle[0] = points[np.argmin(sum_points)]
    rectangle[2] = points[np.argmax(sum_points)]
    
    
    #bottom right will have the smallest difference
    #top left will have the largest difference.
    diff_points = np.diff(points, axis=1)
    rectangle[1] = points[np.argmin(diff_points)]
    rectangle[3] = points[np.argmax(diff_points)]
    
    
    # return order of co-ordinates.
    return rectangle




# set four points.
def set_four_points(image, points):
    # obtain order of points and unpack.
    rectangle = arrange_points(points)
    (top_left,top_right,bottom_right,bottom_left) = rectangle
    # let's compute width of the rectangle.
    # using formular for distance between two points
    left_height = np.sqrt(((top_left[0]-bottom_left[0])**2) + ((top_left[1]-bottom_left[1])**2))
    right_height = np.sqrt(((top_right[0]-bottom_right[0])**2) + ((top_right[1]-bottom_right[1])**2))
    top_width = np.sqrt(((top_right[0]-top_left[0])**2) + ((top_right[1]-top_left[1])**2))
    bottom_width = np.sqrt(((bottom_right[0]-bottom_left[0])**2) + ((bottom_right[1]-bottom_left[1])**2))
    
    maxheight = max(int(left_height), int(right_height))
    maxwidth  = max(int(top_width), int(bottom_width))
    
    destination = np.array([
        [0,0],
        [maxwidth -1,0],
        [maxwidth -1, maxheight-1],
        [0, maxheight - 1]], dtype = "float32")
    
    
    matrix = cv2.getPerspectiveTransform(rectangle, destination)
    warped = cv2.warpPerspective(image, matrix, (maxwidth,maxheight))
    
    return warped   

######  Give The Birds Eye View.
since we've set-up the functions, we use it to obtain the top-down view of the image. The paper here is the scan of the original image while the warped is the scan of the gray scaled image which is more important to perform the subsequent operations.

In [14]:
# get the scanned image of the answer sheet.
paper = set_four_points(orig_image, doc_contour.reshape(4,2))
# get the scanned image of the gray scale which is very much ideal for finding contours.
warped = set_four_points(gray, doc_contour.reshape(4,2))

###### Quick Peek.
Check out the scanned image, as compared to the initial original image.

In [15]:
cv2.imshow("paper_scanned", paper)
cv2.imshow("image", warped)
cv2.waitKey(0)

-1

###### Threshold The image.
The nextstep is to obtain the image threhold, i used the thresh otsu which is much more lighter than some other thresholds. The iidea behind the threshold binary inverse which i included is the invert the otsu thresholding which would help us obtain a more focused view of the contents of the document as compared with the OTSU Threshold alone. This of course would help us to focus on the options.

In [18]:
# get the black and white scan to the threshold.
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

###### Quick Peek.
Let's preview the threshold application.

In [20]:
cv2.imshow("paper_scanned", warped)
cv2.imshow("image", thresh)
cv2.waitKey(0)

True

######  Find Options Contours Only.
Next crucial step is to obtain the options contours in the image. To do this, we find contours using the cv2.findContours(), Next we use imutils to grab the contours, i initialized a list to save my question_contours. The next step is to loop through all the contours and pick only those contours that meet my custom conditions. I'm not a magician, i used trial by error method to obtain those conditions under my IF statement. To even be able to specify conditions, i needed to obtain some reasonable measurements from all the contours. That's where my cv2.boundingRect() comes in which outputs, the position and dimension(width and height) of the input contour.

###### >> Finally, i save these contours in my question_contours list.

In [21]:
#find contours in the thresholded image
contours = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
# grab the contours
contours = imutils.grab_contours(contours)
# initialize a list to secure the contours of the options
question_contours = []

#loop through the contours to specifically select the options part alone.
for contour in contours:
    # get x,y, positions and width and height of box.
    (x,y,w,h)  = cv2.boundingRect(contour)
    # print(x,y,w,h) # if you like to see.
    ar = w/float(h) # grab your true aspect ratio of box.
    
    if w>11 and h>11 and ar>1.5:
        question_contours.append(contour)

###### Quick Peek
Let's see the contours as well as how well the conditions worked to trap options ONLY.

In [24]:
sample = paper.copy() # get a copy of the paper script to avoid tampering
cv2.drawContours(sample,question_contours,-1,(225,0,0),2)
# draw all found contours to check if your proposed constraints are very rigid and strict.
# show the image.
cv2.imshow("all options", sample)
cv2.waitKey(0)
cv2.imwrite("output8.jpg", sample)

True

######  Arrange Contours.
After grabbing all the options, the next step is to arrange the contours from top to bottom, this helps to to easily track the contour from each of the option. To understand this futher, the first option A in question 1 is "contour 1", and the last "option E" in "question 10" is "contour 50".
If you take a look at the number of contours in the question contours list, you'll find out that it is 50, each representing one option.

In [25]:
# arrange contours as top-to-bottom.
question_contours = contour_manager.sort_contours(question_contours, method = "top-to-bottom")[0]

###### Find Shaded Options and Mark Script.
To start the code, i first initialized the correct answers to zero, Next, i pick the first question as well as the range of 5 contours which contains the options. Next step is to arrange the contours selecting them 5 each. To do this i used the imutils's contour function which i imported as contour_manager. In the next line, i initialized shaded to zero.

Now that the contours are all set, I worked on all 50 options. To do this, i created another for loop to work on all the 5 options in each question as ordered by the first for loop. The point is, The first for loop loops through each question, The second for loop in it loops through each option in the question of interest.

In looping through the options, i looked for the most white option. What i mean is, if you take a deep look at the thresholded image we are currently working with, you'll discover that the option with the most white background is the shaded one. To obtain this whitest option, i used a black background to mask the image with only the option of interest excluded. i then use the black mask which has a hole (the option of interest) to cover the image. With this, i am then able to count and determine how white the option is.

Eventually, i save the position of the option with the whitest background which of course is the shaded option. After getting this, i am able to extract the option position in that particular question and then compare with my answer key.

In the next line of code, where i have the If statement, that's where i do the scoring and marking. If the shaded option corresponds with the answer key, i use a green marker and increase score by one.
Eventually, After all said and done, i write a text to count in percentage the total score and finally print it out.

######  Code In.

In [27]:
fast_sample=paper.copy()
correct = 0 # initialize score to zero
for (question, index) in enumerate(np.arange(0,len(question_contours),5)): # 5 options so 5 contours for each line
    contours = contour_manager.sort_contours(question_contours[index:(index + 5)])[0] #picking the 5 contours.
    #print(contours)
    
    shaded = None # initialize shaded option to zero
    cv2.drawContours(fast_sample,contours,-1,(0,225,0),2)
    cv2.imshow("all options", fast_sample)
    cv2.imwrite("output{}.jpg".format(index+100), fast_sample)
    cv2.waitKey(0)
    # for each option in the first 5 contours.
    for (option_position, contour) in enumerate(contours):
        mask = np.zeros(thresh.shape, dtype = "uint8") # give us a mask exactly the size of the answer sheet.
        cv2.drawContours(mask, [contour], -1, (225,225,225), -1) #draw a white box around the option.
        mask = cv2.bitwise_and(thresh, thresh, mask = mask)# mask whole script except option
        total = cv2.countNonZero(mask)# how white is the option?.
        if shaded is None or total > shaded[0]: #most white = most shaded.
            shaded = (total,option_position) #save shaded position
    color  = (0,0,225) #marking ink is red unless:
    correct_option = ANSWER_KEY[question] # extract correct option from answers key.
    if correct_option == shaded[1]: # use green ink if correct answer == shaded option
        color = (0,225,0)
        correct = correct+1 # increase score by 1
    cv2.drawContours(paper, [contours[correct_option]], -1, color, 3) # draw contours for this question.
text = " score is {}%".format(correct*10) # collate remark after marking
script = paper.copy()# dont spoil the original paper with remarks use a copy
cv2.putText(script, text, (100,70), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.9, (0,0,0), 2) # wtite remark
cv2.imshow("stuff", script)# show the whole stuff
cv2.waitKey(0)
cv2.imwrite("output_finale.jpg", script)

True

###### The End
To use this code on different omr sheet types the Parameters to tweak are:

1) The resizing of the image

2) The Threshold parameters

3) The Canny parameters to detect edges

4) The conditions required to find the options.